In [1]:
url = "https://api.go1.com/v2/learning-objects?limit=50&scroll=true"
token = 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJhdWQiOiJiNTJkYjdmNWM2ZmI3MjIzZWM5MWJlZmU0MzJiNjExNmM2N2U2YmFhIiwianRpIjoiNTAwZWM0NGFjZTBjYjM2ZWJiNzFhNjYyMmNkNGIxNmYzNmZjNDJlYzA5MWE3ZDA5MGRkZWY5NmNiOTM0ZGZiNGE0MzFlZmMzZWY2ZjhkNDAiLCJpYXQiOjE2NTI2OTkyOTAsIm5iZiI6MTY1MjY5OTI5MCwiZXhwIjoxNjUyNzQyNDkwLCJzdWIiOiIxMDkyMjQ4NiIsInNjb3BlcyI6WyJhY2NvdW50LnJlYWQiLCJlbnJvbGxtZW50LnJlYWQiLCJsby5yZWFkIiwicG9ydGFsLnJlYWQiLCJ1c2VyLmxvZ2luLm1lIiwidXNlci5sb2dpbiIsInVzZXIucmVhZCIsImdyb3VwLnJlYWQiLCJ3ZWJob29rLnJlYWQiLCJ1c2VyLndyaXRlIiwiZW5yb2xsbWVudC53cml0ZSIsIndlYmhvb2sud3JpdGUiLCJncm91cC53cml0ZSIsImxvLndyaXRlIiwicG9ydGFsLndyaXRlIl19.pIPhhlCSmgnNPzpdheGTguVgg5fI7B3Ie8pO1Slm0DDxmUNGLyzWLbUtR7v9ejscccaHaS5UYxMphEzW7b-bA87KlvlfnxEY71bAbuMl_2qwbPXoNxgMSZF5pIk51hU1U2ybGm4urfUhPaT6GyH9JPXhJxhewXI0Tkhq22rupmm_Wh-0sb3RwTGzNaJiThP7_PccaJyzMYwPE00GXAZnbL8SIA8ItW1ACk1H0muW3HHcEZyOcgom_vQPC44367NMMW92KWMkAd5KylGW3fuWdh_QbzErAmYb_plOEKy3nTabLDyFOZ_bkLsAXIxtpcIYMuCkk-EgOVH8E1q4BTZhhg'
data_path = './courses.json'

In [11]:
import requests
import time
from tqdm.notebook import tqdm
import json

headers = {'Authorization': token}

In [3]:
def retry(exception=Exception, max_retry=5, wait=1):

    def wrapper(func):
        tries = 0
        
        def __inner__(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except exception:
                nonlocal tries
                tries += 1
                if tries > max_retry:
                    raise exception
                time.sleep(wait)
                __inner__(*args, **kwargs)     
        
        return __inner__

    return wrapper

In [4]:
@retry(wait=3)
def get(*args, **kwargs):
    rep = requests.get(*args, **kwargs)
    if rep.status_code != 200:
        raise Exception
    return rep

In [5]:
def go1_data(url, headers):
    resp = get(url, headers=headers).json()
    if len(resp["hits"]) == 0:
        return
    yield from resp["hits"]
    yield from go1_data(url, headers)

In [6]:
resp = get(url, headers=headers).json()
scroll_id = resp["_scroll_id"]
new_url = f"{url}&scrollId={scroll_id}"
total = resp["total"]

In [7]:
result = [x for x in resp["hits"]]
for x in tqdm(go1_data(new_url, headers),total=total):
    result.append(x)

  0%|          | 0/80220 [00:00<?, ?it/s]

In [12]:
with open(data_path, 'w') as json_file:
    json.dump(result, json_file)